In [1]:
import numpy as np

def pieceToVector(piece: str, isBlackTurn: bool = False):
    pieceMap = {
        "r": 0,
        "n": 1,
        "b": 2,
        "q": 3,
        "k": 4,
        "p": 5
    }

    vector = np.zeros(7)
    vector[pieceMap[piece.lower()]] = 1

    if ((not isBlackTurn) and piece.isupper()) or (isBlackTurn and piece.islower()):
        vector[6] = 1

    return vector

def fenToMatrix(fen: str, isBlackTurn: bool = False):
    matrix = np.zeros((8,8,7))
    x = 0
    y = 0
    for char in fen:
        if char.isnumeric():
            x += int(char)
        elif char == "/":
            x = 0
            y += 1
        elif char == " ":
            break
        else:
            matrix[x,y,:] = pieceToVector(char, isBlackTurn)
            x += 1
    return matrix


def coordinateToVector(coordinate):
    pieceMap = {
        "a": 0,
        "b": 1,
        "c": 2,
        "d": 3,
        "e": 4,
        "f": 5,
        "g": 6,
        "h": 7,
    }

    vector = np.zeros(64)
    vector[(8* (int(coordinate[1]) - 1)) +  pieceMap[coordinate[0]]] = 1

    return vector

def uciToVector(uci):
    fromVec = coordinateToVector(uci[:2])
    toVec = coordinateToVector(uci[2:])

    return np.kron(fromVec,toVec)
    

def getGameResult(headers):

    if headers["Termination"] == "Time forfeit":
        return None
    
    result = headers["Result"]

    if result == "1-0":
        return 1
    if result == "1/2-1/2":
        return 0.5
    if result == "0-1":
        return 0


In [50]:
from chess import pgn
import pandas as pd


gamesLimit = 1000000

boards = []
evaluations = []
gameResults = []

#turnOrientedBoards = []
#moves = []


index = 0

with open("lichess_db_standard_rated_2023-04.pgn","r") as chessData:

    game = pgn.read_game(chessData)


    while(game):
        if index > gamesLimit:
            break
        board = game.board()
        headers = game.headers
        result = getGameResult(headers)

        if int(headers["BlackElo"]) < 1200 or int(headers["WhiteElo"]) < 1200:
            game = pgn.read_game(chessData)
            continue

        isBlackTurn = False
        for move in game.mainline():
            boardMatrix = fenToMatrix(board.board_fen())

            #turnBoard = fenToMatrix(board.board_fen(), isBlackTurn)
            #isBlackTurn = not isBlackTurn


            eval = move.eval()
            if eval is None:
                break
            else:
                eval = eval.white().score()


            #uci = move.uci()
            #moveVector = uciToVector(uci)

            
            boards.append(boardMatrix)
            evaluations.append(eval)
            gameResults.append(result)
            #turnOrientedBoards.append(turnBoard)
            #moves.append(moveVector)

            board.push(move.move)

        game = pgn.read_game(chessData)
        index += 1


KeyboardInterrupt: 

In [51]:
evaluationData = pd.DataFrame({"Board": boards, "Evaluation": evaluations, "Result": gameResults})
#movingData = pd.DataFrame({"Board": turnOrientedBoards, "Move": moves})

In [52]:
evaluationData.to_pickle("evaluation_data-2023-4.pkl")
#movingData.to_pickle("moves_data-2023-4.pkl")

In [1]:
import pandas as pd
import numpy as np
movingData = pd.read_pickle("moves_data-2023-4.pkl")

boardData = np.array([board for board in movingData["Board"]])
moveData = np.array([move for move in movingData["Move"]])



In [13]:
boardData = np.array([board for board in movingData["Board"]])
moveData = np.array([move for move in movingData["Move"]])

In [24]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


boardShape = (8,8,7)


movePredictorModel = keras.Sequential([
    layers.Input(boardShape),
    layers.Conv2D(56,5, activation="tanh"),
    layers.Conv2D(112,3, activation="tanh"),
    layers.Conv2D(224,2, activation="tanh"),
    layers.Flatten(),
    layers.Dense(256,activation="tanh"),
    layers.Dense(256,activation="tanh"),
    layers.Dense(512,activation="tanh"),
    layers.Dense(512,activation="tanh"),
    layers.Dense(64*64, activation= "softmax")
])

print(movePredictorModel.summary())

movePredictorModel.compile(
    optimizer= "adam",
    loss= "categorical_crossentropy",
    metrics= ['accuracy']
)
#movePredictorModel.load_weights("movePredictor-1.h5")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_18 (Conv2D)          (None, 4, 4, 56)          9856      
                                                                 
 conv2d_19 (Conv2D)          (None, 2, 2, 112)         56560     
                                                                 
 conv2d_20 (Conv2D)          (None, 1, 1, 224)         100576    
                                                                 
 flatten_6 (Flatten)         (None, 224)               0         
                                                                 
 dense_30 (Dense)            (None, 256)               57600     
                                                                 
 dense_31 (Dense)            (None, 256)               65792     
                                                                 
 dense_32 (Dense)            (None, 512)              

In [44]:
movePredictorModel.fit(
    x = boardData,
    y = moveData,

    epochs= 10,
    batch_size = 128,

    shuffle= True,
    validation_split= 0.2
)

Epoch 1/10
4205/4205 [==============================] - 216s 51ms/step - loss: 4.1617 - accuracy: 0.1414 - val_loss: 4.6167 - val_accuracy: 0.1197
Epoch 2/10
4205/4205 [==============================] - 208s 50ms/step - loss: 4.1611 - accuracy: 0.1411 - val_loss: 4.6078 - val_accuracy: 0.1263
Epoch 3/10
4205/4205 [==============================] - 207s 49ms/step - loss: 4.1637 - accuracy: 0.1409 - val_loss: 4.6127 - val_accuracy: 0.1271
Epoch 4/10
4205/4205 [==============================] - 207s 49ms/step - loss: 4.1587 - accuracy: 0.1413 - val_loss: 4.6069 - val_accuracy: 0.1266
Epoch 5/10
4205/4205 [==============================] - 207s 49ms/step - loss: 4.1667 - accuracy: 0.1402 - val_loss: 4.6303 - val_accuracy: 0.1244
Epoch 6/10
4205/4205 [==============================] - 207s 49ms/step - loss: 4.1694 - accuracy: 0.1395 - val_loss: 4.6182 - val_accuracy: 0.1272
Epoch 7/10
1461/4205 [=========>....................] - ETA: 2:09 - loss: 4.1040 - accuracy: 0.1447

KeyboardInterrupt: 

In [43]:
movePredictorModel.save("movePredictor.h5")

In [48]:


board = boardData[10]

print(board)
prediction = movePredictorModel.predict(np.asarray([board]))[0]

max = np.max(prediction)
print(np.max(prediction))

print(np.argmax(prediction))

prediction = (prediction > max/4)*prediction

np.set_printoptions(threshold=np.inf)

print(prediction)

[[[1. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1.]
  [1. 0. 0. 0. 0. 0. 1.]]

 [[0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 1. 0. 0. 0. 0. 1.]]

 [[0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 1. 0. 0. 0. 1.]]

 [[0. 0. 0. 1. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 1. 0. 0. 1.]]

 [[0. 0. 0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 1.]
  [0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 1.

In [1]:
import pandas as pd
import numpy as np
evaluationData = pd.read_pickle("evaluation_data-2023-4.pkl")

In [14]:
evaluationData["Result"] = evaluationData["Result"].shift(1)

In [15]:
print(len(evaluationData))

15558996


In [16]:
evaluationData.dropna(subset = ["Result"], inplace = True)

In [17]:
evaluationData.to_pickle("evaluation_data-2023-4.pkl")

In [2]:
resultBoardData = np.array([board for board in evaluationData["Board"]])
results = np.array([result for result in evaluationData["Result"]])


In [3]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


boardShape = (8,8,7)

boardEvaluationModel = keras.Sequential([
    layers.Input(boardShape),
    layers.Conv2D(28,3),
    layers.Conv2D(56,3),
    layers.Conv2D(112,3),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128),
    layers.Dense(64),
    layers.Dense(1)
])

print(boardEvaluationModel.summary())

boardEvaluationModel.compile(
    optimizer= "RMSprop",
    loss= "mean_squared_error",
    metrics= ['accuracy']
)


boardEvaluationModel.load_weights("evaluator.h5")

2023-05-29 08:05:31.718427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 4, 4, 56)          9856      
                                                                 
 conv2d_1 (Conv2D)           (None, 2, 2, 112)         56560     
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 224)         100576    
                                                                 
 flatten (Flatten)           (None, 224)               0         
                                                                 
 dense (Dense)               (None, 256)               57600     
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8

In [4]:
boardEvaluationModel.fit(
    x = resultBoardData,
    y = results,

    epochs= 10,
    batch_size = 128,

    shuffle= True,
    validation_split= 0.2
)

Epoch 1/10
37328/60522 [=================>............] - ETA: 3:42 - loss: 0.6168 - accuracy: 0.4774

KeyboardInterrupt: 

In [26]:
boardEvaluationModel.save("evaluator.h5")

: 